In [1]:
#!/bin/bash
# !curl -L -o ./online-retail-dataset.zip\
  # https://www.kaggle.com/api/v1/datasets/download/lakshmi25npathi/online-retail-dataset
# from https://www.kaggle.com/datasets/lakshmi25npathi/online-retail-dataset

In [2]:
import io
from zipfile import ZipFile
def read_zip(zip_fn, extract_fn=None):
    zf = ZipFile(zip_fn)
    if extract_fn: return zf.read(extract_fn)
    else: return {name:zf.read(name) for name in zf.namelist()}

In [3]:
import pandas as pd

df = pd.read_excel(
    io.BytesIO(read_zip(r'./online-retail-dataset.zip', 'online_retail_II.xlsx'))
)
print(df.shape)
df.sample(5)

(525461, 8)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
167478,505209,21534,DAIRY MAID LARGE MILK JUG,1,2010-04-20 17:25:00,4.95,14081.0,United Kingdom
475552,534166,22326,ROUND SNACK BOXES SET OF4 WOODLAND,12,2010-11-21 13:15:00,2.95,17702.0,United Kingdom
166693,505177,21913,VINTAGE SEASIDE JIGSAW PUZZLES,2,2010-04-20 13:43:00,3.75,13042.0,United Kingdom
490890,535344,22592,CARDHOLDER HOLLY WREATH METAL,4,2010-11-25 16:36:00,3.75,15266.0,United Kingdom
442163,531489,85049A,TRADITIONAL CHRISTMAS RIBBONS,2,2010-11-08 15:02:00,2.51,NaN,United Kingdom


In [16]:
# df.info()
df = df[
    df.InvoiceDate.dt.year.isin([2010,2011])
]
df = df.rename(columns={'Customer ID':'CustomerID'})
df.shape
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'CustomerID', 'Country', 'TotalPrice'],
      dtype='object')

In [14]:
# df.columns
df = df.dropna(subset=['CustomerID'])
df = df[df['Quantity'] > 0]
df = df[df['Price'] > 0]

df['TotalPrice'] = df['Quantity'] * df['Price']


In [17]:
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'Invoice': 'nunique',
    'TotalPrice': 'sum'
})

rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm = rfm.reset_index()

In [18]:
rfm

,CustomerID,Recency,Frequency,Monetary
0,12346.0,165,6,259.36
1,12347.0,3,2,1323.32
2,12348.0,74,1,222.16
3,12349.0,43,3,2671.14
4,12351.0,11,1,300.93
...,...,...,...,...
4194,18283.0,18,6,641.77
4195,18284.0,67,1,461.68
4196,18285.0,296,1,427.00
4197,18286.0,112,1,833.48


In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[['Recency', 'Frequency', 'Monetary']])

In [21]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, random_state=42)
labels_kmeans = kmeans.fit_predict(rfm_scaled)

In [22]:
from sklearn.cluster import AgglomerativeClustering

agg = AgglomerativeClustering(n_clusters=4)
labels_agg = agg.fit_predict(rfm_scaled)


In [23]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

def evaluate(X, labels, name):
    print(f"\n{name}")
    print("Silhouette:", silhouette_score(X, labels))
    print("Calinski-Harabasz:", calinski_harabasz_score(X, labels))
    print("Davies-Bouldin:", davies_bouldin_score(X, labels))


for k in range(2, 8):
    km = KMeans(n_clusters=k, random_state=42)
    labels = km.fit_predict(rfm_scaled)
    print(
        k,
        silhouette_score(rfm_scaled, labels),
        calinski_harabasz_score(rfm_scaled, labels),
        davies_bouldin_score(rfm_scaled, labels)
    )


2 0.5648668674467573 1624.049916033544 0.631748349923151
3 0.5916060285114889 3125.2821976547316 0.636295444557263
4 0.6115198110131461 3651.4238563538606 0.6250863934730335
5 0.6162484572771778 3883.661148244 0.61795581334914
6 0.4917059236041258 4007.7254588563123 0.7097729768445479
7 0.4827040524265239 4216.154438739266 0.6858962464368453


In [24]:
rfm['Cluster'] = labels_kmeans
rfm.groupby('Cluster').mean()


,CustomerID,Recency,Frequency,Monetary
Cluster,,,,
0,15337.750080,41.183699,4.196753,1622.504701
1,15380.417252,222.243731,1.584754,593.885380
2,15101.800000,5.600000,105.200000,196683.554000
3,15139.660714,14.785714,43.089286,26313.220589
